<a href="https://colab.research.google.com/github/hilanistel/STCP_feature_selection/blob/STCP-Motivating-example/TS_method_Mushrooms_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test Reading , R.

In [4]:
from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive')
#load users list and movies list
Mushrooms_data = pd.read_csv("gdrive/My Drive/תזה תואר שני/Mushrooms/Readings.csv",header=None)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
X=Mushrooms_data.drop(21, axis=1) # drop the reading class
TR = [tuple(row) for row in X.values]
Mushrooms_data = [list(row) for row in Mushrooms_data.values]

𝑝(𝐫̃) - The a-priory probability of obtaining the reading 𝐫̃∈𝑅.

In [6]:
import numpy as np
res = set()  
uniqe_readings = [res.add(i) for i in TR  
              if i not in res] 
pr_ = []
for j in res:
  pr_.append((list(map(lambda i: i, TR)).count(j))/len(TR))
pr_ = tuple(pr_) # convert list pr_ to tuple
#pr_

𝑝(𝐫̃|𝑘) - The conditional probability of each reading given a class.

In [7]:
pr_k = {}
for key in range(0,2):
  pr_k.update({key: {}})
  k_readings= []
  for item in Mushrooms_data:
    if (item[-1] == key):
      k_readings.append(tuple(item[:-1]))
  l = []
  for j in res:
    l.append((list(map(lambda i: i, k_readings)).count(j))/len(k_readings))
  pr_k[key] = tuple(l)
    
#print(pr_k)
#pr_k = {"Negative": (0.035,0.015,0.105,0.045,0.140,0.060,0.420,0.180), "Positive": (0.105,0.245,0.105,0.245,0.045,0.105,0.045,0.105)}

𝑝(𝑘|𝐫̃) - The conditional probability of class 𝑘∈𝐾 given the reading 𝐫̃∈𝑅.

In [8]:
pkr_ = {}
for key in range(0,2):
  pkr_.update({key: {}})
  l = []
  for r in res:
    k_class= []
    for item in Mushrooms_data:
        if (tuple(item[:-1]) == r):
          k_class.append(item[-1])
    l.append((list(map(lambda i: i, k_class)).count(key))/len(k_class))
  pkr_[key] = tuple(l)
    
#print(pkr_)
#pkr_ = {"Negative": (0.289,0.070,0.550,0.183,0.792,0.411,0.919,0.677), "Positive": (0.711,0.930,0.450,0.817,0.208,0.589,0.081,0.323)}
#check if every reading is 1 prob
#for i,r in enumerate(res):
#prob_r = 0
#for key in range(0,7):
#  prob_r = prob_r + pkr_[key][i]
#print(prob_r)

𝜆𝑘𝑙 - The misclassification error of type (𝑘,𝑙), 𝑘,𝑙∈𝐾, i.e., the cost of classifying an object as class 𝑙 when its true class is 𝑘.

In [9]:
drive.mount('/content/gdrive')
#load users list and movies list
𝜆𝑘𝑙 = pd.read_csv("gdrive/My Drive/תזה תואר שני/Mushrooms/Penalties1.csv",header=None)
𝜆𝑘𝑙 = [list(row) for row in 𝜆𝑘𝑙.values]
#𝜆𝑘𝑙 = [[0,50],[50,0]]
𝜆𝑘𝑙

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


[[0, 300], [500, 0]]

𝐾 - The set of possible classes; 𝐾={1,…,𝑘}.

In [10]:
K = { 0:0 , 1:1}

𝑁 - The set of candidate tests available in a given system; the number of tests is denoted by 𝑛=|𝑁|. 𝑆∈𝑁 is a subset of selected tests called a configuration.

In [11]:
N = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]

In [12]:
# function create to get all N subsets
def sub_lists (l): 
    base = []   
    lists = [base] 
    for i in range(len(l)): 
        orig = lists[:] 
        new = l[i] 
        for j in range(len(lists)): 
            lists[j] = lists[j] + [new] 
        lists = orig + lists 
          
    return lists

subsets =  sub_lists(N)
subsets.remove([])

𝑐𝑖 - The cost of test 𝑖 for any 𝑖∈𝑁.


In [13]:
ci = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

R(S) - given a configuration 𝑆⊂𝑁 and test reading R,
We define function **RS(S,R)** to return the set of all its signatures.



In [14]:
def RS(S,R):
  R_S = []
  for j in R:
    r = []
    for i in S:
        r.append(j[i-1])
    if r not in R_S:
      R_S.append(r)
  #print("RS")
  #print(R_S)
  return (R_S)

# R(S) for 𝑆={1,2}
#RS((1,2), TR)

Q(S,r) - given a configuration 𝑆⊂𝑁 and signature r,  We defined **function QSr(S,r)**  to return the set of all the possible readings from which signature 𝐫 can be obtained when the configuration is 𝑆. 𝑄(𝑆,𝐫)={𝐫̃∈𝑅:𝑟̃𝑖=𝑟𝑖 ∀𝑖∈𝑆}.



In [15]:
def QSr(S,r):
  Q_S = []
  for j in res:
    rj = []
    for i in S:
      rj.append(j[i-1])
    if tuple(rj) == r :
        Q_S.append(j)
  #print("QS")
  #print(Q_S)
  return (Q_S)


#QSr((1,2,3,4),(2,5,5,1))

**Formulation (1)** 𝑝(𝑘)= Σ𝑝(𝑘|𝐫̃)𝑝(𝐫̃) ; 𝐫̃∈𝑅 - the prior probability of each class.

In [16]:
def Pk(k):
  total_prob = 0 
  for index_r_, r_ in enumerate(pr_):
    for index_kr_, kr_ in enumerate(pkr_[k]):
      if index_r_ == index_kr_:
        total_prob += r_ * kr_
  #print("Pk")
  #print(total_prob)
  return (total_prob)

# Pk(k) for k = 1
#Pk(1)

In [17]:
Pk_dict ={}
for k in K:
  Pk_dict.update({k : {}})
  Pk_dict[k] = Pk(k)

**Formulation (3)** Ps(𝐫|𝑘)=ΣP(𝐫̃|𝑘) ;
𝐫̃∈𝑄(𝑆,𝐫) - the probability of a signature given class 𝑘.







In [18]:
# We created nested dictionary : first key is a class , second key is a reading, value is the conditional probability of the reading given the class.
# for example: {"Negative": {"(0,0,0)": 0.035 }}
dict_k_r_prk = {}
for key in pr_k:
  dict_k_r_prk.update({key: {}})
  for index_i, i in enumerate(res):
    dict_k_r_prk[key].update({i:{}})
    for index_j, j in enumerate(pr_k[key]):
      if index_j == index_i:
        dict_k_r_prk[key][i] = j 
#print(dict_k_r_prk)

In [19]:
# Formulation (3) implementation
def PSrk1(k,QS):
  total_prob = 0
  for r_ in QS:
    total_prob += dict_k_r_prk[k][r_]
  #print("PSkr")
  #print(total_prob)
  return (total_prob)

dict_QS ={}
def PSrk(k,QS):
  if tuple(QS) in dict_QS.keys():
    if k not in dict_QS[tuple(QS)].keys():
      dict_QS[tuple(QS)].update({k:{}})
      dict_QS[tuple(QS)][k]=PSrk1(k,tuple(QS))
  else:
      dict_QS.update({tuple(QS):{}})
      dict_QS[tuple(QS)].update({k:{}})
      dict_QS[tuple(QS)][k]=PSrk1(k,tuple(QS))
  return (dict_QS[tuple(QS)][k])

#print(PSrk(2,[(2,5,5,1,3,4,1,1,4,5,4,4)]))
#print(PSrk(2,[(2,5,5,1,3,5,2,1,5,4,5,3)])/4)
#print(PSrk(1,[(1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5)]))
#print(PSrk(2,[(2,5,4,5,2,5,1,1,5,1,5,1)])/6)

**Formulation (4)** Ps(𝐫)=ΣP(𝐫̃) ; 𝑟̃∈𝑄(𝑆,𝐫) - the prior probability of signature 𝐫∈𝑅(𝑆).

In [20]:
# We created a dictionary : key is a reading, value is the a-priory probability of obtaining the reading 𝐫̃∈𝑅.
# for example: {"(0,0,0)": 0.067 }
dict_r_pr_ = {}
for index_i, i in enumerate(res):
    dict_r_pr_.update({i:{}})
    for index_j, j in enumerate(pr_ ):
      if index_j == index_i:
       dict_r_pr_[i] = j 
#print(dict_r_pr_)

In [21]:
# Formulation (4) implementation
def PSr(QS):
  total_prob = 0
  for r_ in QS:
    total_prob += dict_r_pr_ [r_]
  #print("PSr")
  #print(total_prob)
  return (total_prob)


# PSr(S,r) for 𝑆={1,2} and r = (0,0)
#PSr((1,2,3,4,5,6,7,8,9,10),(1, 1, 1, 2, 5, 2, 3, 3, 5, 5))

**Formulation (5)** Ps(𝑘|𝐫)=Ps(𝐫|𝑘)∙𝑝(𝑘) / Ps(𝐫) - the a-posteriori probability that the class is 𝑘∈𝐾 given that the signature is 𝐫∈𝑅(𝑆).

**Formulation (6)** 𝐸s(𝐫|𝑙)= Σ𝜆𝑘𝑙∙Ps(𝑘|𝐫) ; 𝑘∈𝐾 - Consider a given configuration 𝑆⊂𝑁. The expected classification error cost for signature 𝐫 of 𝑆 if it is mapped to class 𝑙

In [22]:
def ESrl(l,Psr,Qs):
  sum = 0 
  for k in K:
        sum += 𝜆𝑘𝑙[K[k]][K[l]] * (PSrk(k,Qs) * Pk_dict[k] / Psr)
  #print("ESrl")
  #print(sum)
  return sum
  
# ESrl(S,r,l) for 𝑆={1,2} and r = (0,0) and l="Positive"
#ESrl((1,2),(0,0),1)

**Formulation (7)** 𝑙𝑆∗(𝐫)=argmin{𝐸𝑆(𝐫|𝑙)} ; 𝑙∈𝐾.

In [23]:
def lS(S,r):
  min = 100
  for l in K:
          Qs  = QSr(S,r)
          Psr = PSr(S,r,Qs)
          ESRL = ESrl(l,Psr,Qs)
          if (ESRL < min):
            min = ESRL
            k = l
  #print("lS")
  #print(k)
  return k

# lS(r) for r = (0,0,0)
#lS((1,2),(0,0))

**Formulation (8)** 𝐸𝑆∗(𝐫)=min{𝐸𝑆(𝐫|𝑙)} ; 𝑙∈𝐾 - The minimum expected classification error cost for signature *𝐫*

In [24]:
def ES(Qs,Psr):
  #print("ES")
  min = 100
  for l in K:
          ESRL = ESrl(l,Psr,Qs)
          if (ESRL < min):
            min = ESRL
  #print(min)
  return min

# ES(r) for r = (0,0,0)
#ES((1,2),(0,0)
#ES([],[])

**The TS method**

In [25]:
 dict_N_x = {}
def N_x(x,TL):
  dict_N_x = {}
  dict_N_x.update({tuple(x):{}}) # step, subset, diff
  for n in subsets:
    if len(n)-len(x) == 1:
      notsame = 0
      for i in n:
       if i not in x:
            notsame = notsame + 1
            diff = i
      if notsame == 1:
        if diff not in TL["ADD"]:
          if "ADD" not in dict_N_x[tuple(x)].keys():
            dict_N_x[tuple(x)].update({"ADD":{}})
          dict_N_x[tuple(x)]["ADD"].update({tuple(n):{}})
          dict_N_x[tuple(x)]["ADD"][tuple(n)] = diff
    if len(x)-len(n) == 1:
      notsame = 0
      for i in x:
        if i not in n:
            notsame = notsame + 1
            diff = i
        if notsame == 1:
          if diff not in TL["REMOVE"]:
            if "REMOVE" not in dict_N_x[tuple(x)].keys():
              dict_N_x[tuple(x)].update({"REMOVE":{}})
            dict_N_x[tuple(x)]["REMOVE"].update({tuple(n):{}})
            dict_N_x[tuple(x)]["REMOVE"][tuple(n)] = diff
    if len(x)-len(n) == 0:
      notsame1 = 0
      notsame2 = 0
      for i in x:
        if i not in n:
            notsame1 = notsame1 + 1
            diff_REMOVE = i
      for j in n:
        if j not in x:
            notsame2 = notsame2 + 1
            diff_ADD = j 
        if notsame1 == 1 and notsame2 == 1:
          if [diff_ADD,diff_REMOVE] not in TL["SWAP"]:
            if "SWAP" not in dict_N_x[tuple(x)].keys():
              dict_N_x[tuple(x)].update({"SWAP":{}})
            dict_N_x[tuple(x)]["SWAP"].update({tuple(n):{}})
            dict_N_x[tuple(x)]["SWAP"][tuple(n)] = [diff_ADD,diff_REMOVE]
  return(dict_N_x)

**Formulation (9)** min𝑆⊆𝑁{Σ𝑝𝑆(𝐫)𝐸𝑆∗(𝐫) ; 𝐫∈𝑅(𝑆) +Σ𝑐𝑖 ; 𝑖∈𝑆} - The STCP formulated mathematically

In [27]:
def stcp(x):
      test_cost = 0 
      classification_error = 0 
      for r in RS(x,TR):
        Qs = QSr(tuple(x),tuple(r))
        Psr = PSr(Qs)
        classification_error += Psr * ES(Qs,Psr)
      for i in x:
        test_cost += ci[i-1]
      total_error = classification_error + test_cost
      return(total_error)

In [28]:
def Tabu():
  x = []
  x_ = []
  v_ = stcp(x)
  Nx = []
  TL = {"ADD":[()],"REMOVE":[], "SWAP":[]}
  for i in range(0,90): #num of iterations
    m = {"ADD":[],"REMOVE":[], "SWAP":[]}
    v = 100
    v__ = 0
    dict_N_x = N_x(x,TL)
    print(dict_N_x)
    print(TL)
    for i in dict_N_x.keys():
      for j in dict_N_x[i]:
        for x__ in dict_N_x[i][j]:
          if dict_N_x[i][j][x__] not in TL[j]:
            v__ = stcp(x__)
            #print(v__)
            #print(v)
            if v__ < v:
              #print(x__)
              v = v__
              y = x__
              index1 = i
              index = j
              if dict_N_x[index1][index][y] not in TL[index]:
                TL[index].append(dict_N_x[index1][index][y])
    x = y
    if v < v_:
      x_ = x
      v_ = v
    if (len(TL[index]) == 5):
        TL[index].remove(TL[index][0])
    print(v_)
    print(x_)

Tabu()

{(): {'ADD': {(1,): 1, (2,): 2, (3,): 3, (4,): 4, (5,): 5, (6,): 6, (7,): 7, (8,): 8, (9,): 9, (10,): 10, (11,): 11, (12,): 12, (13,): 13, (14,): 14, (15,): 15, (16,): 16, (17,): 17, (18,): 18, (19,): 19, (20,): 20, (21,): 21}}}
{'ADD': [()], 'REMOVE': [], 'SWAP': []}
8.385524372231206
(5,)
{(5,): {'SWAP': {(1,): [1, 5], (2,): [2, 5], (3,): [3, 5], (4,): [4, 5], (6,): [6, 5], (7,): [7, 5], (8,): [8, 5], (9,): [9, 5], (10,): [10, 5], (11,): [11, 5], (12,): [12, 5], (13,): [13, 5], (14,): [14, 5], (15,): [15, 5], (16,): [16, 5], (17,): [17, 5], (18,): [18, 5], (19,): [19, 5], (20,): [20, 5], (21,): [21, 5]}, 'ADD': {(2, 5): 2, (3, 5): 3, (5, 6): 6, (5, 7): 7, (5, 8): 8, (5, 9): 9, (5, 10): 10, (5, 11): 11, (5, 12): 12, (5, 13): 13, (5, 14): 14, (5, 15): 15, (5, 16): 16, (5, 17): 17, (5, 18): 18, (5, 19): 19, (5, 20): 20, (5, 21): 21}}}
{'ADD': [(), 1, 4, 5], 'REMOVE': [], 'SWAP': []}
4.954209748892477
(5, 19)
{(5, 19): {'REMOVE': {(1,): 5, (2,): 5, (3,): 5, (4,): 5, (5,): 19, (6,): 5, (7

In [29]:
print("the solution is:%s", "(5, 19, 20, 21)")
print("the solution total error is :%s", stcp((5, 19, 20, 21)))

the solution is:%s (5, 19, 20, 21)
the solution total error is :%s 4.0
